In [1]:
%pip install -r ../requirements.txt

  Using cached usaddress-0.5.11-py3-none-any.whl.metadata (6.9 kB)
ERROR: Could not find a version that satisfies the requirement sqlite (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for sqlite
Note: you may need to restart the kernel to use updated packages.


### Setup

In [2]:
import pandas as pd
import numpy as np
import sqlite3
from uszipcode import SearchEngine

/Users/thomaschamberlain/Projects/green_industrial_policy/green_industrial_policy/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


AttributeError: module 'sqlalchemy_mate' has no attribute 'ExtendedBase'

In [3]:
# Define test path
test_assistance_file_path = "../data/raw/Federal Spending/FY2008_All_Assistance_Full_20240708/FY2008_All_Assistance_Full_20240708_1.csv"
test_contract_file_path = "../data/raw/Federal Spending/FY2008_All_Contracts_Full_20240708/FY2008_All_Contracts_Full_20240708_1.csv"

In [4]:
# Database connection
db_path = "federal_spending_test.db"
conn = sqlite3.connect(db_path)

In [5]:
# Define relevant columns
assistance_columns_to_include = [
    # Primary Key (transaction level)
    "assistance_transaction_unique_key",

    # Secondary Key (award level)
    "assistance_award_unique_key",
    
    # Spending type columns 
    "assistance_type_code", "assistance_type_description",
    "disaster_emergency_fund_codes_for_overall_award",
    
    # Spending amount columns
    "total_obligated_amount", "total_dollars_obligated", "total_outlayed_amount_for_overall_award",
    "obligated_amount_from_IIJA_supplemental_for_overall_award", "outlayed_amount_from_IIJA_supplemental_for_overall_award",

    # Other spending columns
    "indirect_cost_federal_share_amount", "non_federal_funding_amount", "total_non_federal_funding_amount",
    "face_value_of_loan", "original_loan_subsidy_cost", "total_face_value_of_loan", "total_loan_subsidy_cost",
    
    # Recipient geographic identifiers
    "recipient_address_line_1", "recipient_address_line_2",
    "recipient_city_name", "recipient_county_name", "recipient_state_code", "recipient_state_name", 
    "recipient_zip_code", "recipient_zip_last_4_code", "recipient_zip_4_code", "recipient_country_name", 
    
    # Place of performance geographic identifiers
    "primary_place_of_performance_city_name", "primary_place_of_performance_county_name",
    "primary_place_of_performance_state_name", "primary_place_of_performance_zip_4",
]

contract_columns_to_include = [
    # Primary Key (transaction level)
    "contract_transaction_unique_key",

    # Secondary Key (award level)
    "contract_award_unique_key",

    # Spending type columns
    "award_type", "award_type_code", 
    "disaster_emergency_fund_codes_for_overall_award",

    # Spending amount columns 
    "total_obligated_amount", "total_dollars_obligated", "total_outlayed_amount_for_overall_award",
    "obligated_amount_from_IIJA_supplemental_for_overall_award", "outlayed_amount_from_IIJA_supplemental_for_overall_award",

    # Recipient geographic identifiers
    "recipient_address_line_1", "recipient_address_line_2",
    "recipient_city_name", "recipient_county_name", "recipient_state_code", "recipient_state_name", 
    "recipient_zip_code", "recipient_zip_last_4_code", "recipient_zip_4_code", "recipient_country_name", 
    
    # Place of performance geographic identifiers
    "primary_place_of_performance_city_name", "primary_place_of_performance_county_name",
    "primary_place_of_performance_state_name", "primary_place_of_performance_zip_4",


]

In [6]:
# Define data types
assistance_dtypes = {"assistance_transaction_unique_key": str, 
                                       "assistance_award_unique_key": str,
                                       "assistance_type_code": str,
                                       "assistance_type_description": str,
                                       "disaster_emergency_fund_codes_for_overall_award": str,
                                       "total_obligated_amount": float, 
                                       "total_dollars_obligated": float, 
                                       "total_outlayed_amount_for_overall_award": float,
                                       "obligated_amount_from_IIJA_supplemental_for_overall_award": float, 
                                       "outlayed_amount_from_IIJA_supplemental_for_overall_award": float,
                                       "recipient_address_line_1": str,
                                       "recipient_address_line_2": str,
                                       "recipient_city_name": str, 
                                       "recipient_county_name": str, 
                                       "recipient_state_code": str, 
                                       "recipient_state_name": str, 
                                       "recipient_zip_code": str, 
                                       "recipient_zip_last_4_code": str, 
                                       "recipient_zip_4_code": str, 
                                       "recipient_country_name": str,
                                       "primary_place_of_performance_city_name": str, 
                                       "primary_place_of_performance_county_name": str,
                                       "primary_place_of_performance_state_name": str, 
                                       "primary_place_of_performance_zip_4": str}

contract_dtypes = {"contract_transaction_unique_key": str, 
                                     "contract_award_unique_key": str,
                                     "award_type": str,
                                     "award_type_code": str,
                                     "disaster_emergency_fund_codes_for_overall_award": str,
                                     "total_obligated_amount": float, 
                                     "total_dollars_obligated": float, 
                                     "total_outlayed_amount_for_overall_award": float,
                                     "obligated_amount_from_IIJA_supplemental_for_overall_award": float, 
                                     "outlayed_amount_from_IIJA_supplemental_for_overall_award": float,
                                     "recipient_address_line_1": str,
                                     "recipient_address_line_2": str,
                                     "recipient_city_name": str, 
                                     "recipient_county_name": str, 
                                     "recipient_state_code": str, 
                                     "recipient_state_name": str, 
                                     "recipient_zip_code": str, 
                                     "recipient_zip_last_4_code": str, 
                                     "recipient_zip_4_code": str, 
                                     "recipient_country_name": str,
                                     "primary_place_of_performance_city_name": str, 
                                     "primary_place_of_performance_county_name": str,
                                     "primary_place_of_performance_state_name": str, 
                                     "primary_place_of_performance_zip_4": str}

In [7]:
df_assistance = pd.read_csv(test_assistance_file_path, 
                                usecols = lambda x: x in assistance_columns_to_include,
                                dtype={"assistance_transaction_unique_key": str, 
                                       "assistance_award_unique_key": str,
                                       "assistance_type_code": str,
                                       "assistance_type_description": str,
                                       "disaster_emergency_fund_codes_for_overall_award": str,
                                       "total_obligated_amount": float, 
                                       "total_dollars_obligated": float, 
                                       "total_outlayed_amount_for_overall_award": float,
                                       "obligated_amount_from_IIJA_supplemental_for_overall_award": float, 
                                       "outlayed_amount_from_IIJA_supplemental_for_overall_award": float,
                                       "recipient_address_line_1": str,
                                       "recipient_address_line_2": str,
                                       "recipient_city_name": str, 
                                       "recipient_county_name": str, 
                                       "recipient_state_code": str, 
                                       "recipient_state_name": str, 
                                       "recipient_zip_code": str, 
                                       "recipient_zip_last_4_code": str, 
                                       "recipient_zip_4_code": str, 
                                       "recipient_country_name": str,
                                       "primary_place_of_performance_city_name": str, 
                                       "primary_place_of_performance_county_name": str,
                                       "primary_place_of_performance_state_name": str, 
                                       "primary_place_of_performance_zip_4": str})

In [8]:
df_contract = pd.read_csv(test_contract_file_path, 
                              usecols = lambda x: x in contract_columns_to_include,
                              dtype={"contract_transaction_unique_key": str, 
                                     "contract_award_unique_key": str,
                                     "award_type": str,
                                     "award_type_code": str,
                                     "disaster_emergency_fund_codes_for_overall_award": str,
                                     "total_obligated_amount": float, 
                                     "total_dollars_obligated": float, 
                                     "total_outlayed_amount_for_overall_award": float,
                                     "obligated_amount_from_IIJA_supplemental_for_overall_award": float, 
                                     "outlayed_amount_from_IIJA_supplemental_for_overall_award": float,
                                     "recipient_address_line_1": str,
                                     "recipient_address_line_2": str,
                                     "recipient_city_name": str, 
                                     "recipient_county_name": str, 
                                     "recipient_state_code": str, 
                                     "recipient_state_name": str, 
                                     "recipient_zip_code": str, 
                                     "recipient_zip_last_4_code": str, 
                                     "recipient_zip_4_code": str, 
                                     "recipient_country_name": str,
                                     "primary_place_of_performance_city_name": str, 
                                     "primary_place_of_performance_county_name": str,
                                     "primary_place_of_performance_state_name": str, 
                                     "primary_place_of_performance_zip_4": str})

In [13]:
df_assistance['zip_code_plus_4'] = df_assistance['recipient_zip_code'].astype(str) + df_assistance['recipient_zip_last_4_code']
df_contract['zip_code_plus_4'] = df_contract['recipient_zip_4_code']

In [15]:
# Extract unique ZIP+4 codes from both datasets
unique_zip4_assistance = df_assistance['zip_code_plus_4'].dropna().unique()
unique_zip4_contract = df_contract['zip_code_plus_4'].dropna().unique()

In [18]:
# Combine and get unique ZIP+4 codes
unique_zip4_codes = pd.Series(np.concatenate((unique_zip4_assistance, unique_zip4_contract))).unique()